In [ ]:
# Celda 1: Importar Bibliotecas
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import pandas as pd
from pymongo.mongo_client import MongoClient

# Celda 2: Conexión a MongoDB y Obtención de Datos
client = MongoClient("mongodb://localhost:27019/")
db = client["nombre_de_tu_base_de_datos"]
collection = db["nombre_de_tu_coleccion"]
cursor = collection.find({})
df = pd.DataFrame(list(cursor))

# Celda 3: Preprocesamiento y Modelado
le = LabelEncoder()
df["actions"] = le.fit_transform(df["actions"])
df["team"] = le.fit_transform(df["team"])
df["winner"] = le.fit_transform(df["winner"])

X = df[["gameID", "team", "action_count"]]
y = df["winner"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = RandomForestClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# Celda 4: Visualización con Matplotlib
plt.scatter(X_test["gameID"], predictions, label="Predictions", alpha=0.5)
plt.scatter(X_test["gameID"], y_test, label="Actual", alpha=0.5)
plt.xlabel("gameID")
plt.ylabel("Winner (Encoded)")
plt.legend()
plt.show()

# Celda 5: Visualización con Matplotlib y Datos de MongoDB
# Instala las bibliotecas necesarias en tu entorno de Jupyter
# pip install matplotlib pandas pymongo

# Conecta con MongoDB y carga los datos
client_mongo = MongoClient("mongodb://localhost:27017/")
db_mongo = client_mongo["tu_base_de_datos"]
collection_mongo = db_mongo["predicciones"]
data_from_mongo = list(collection_mongo.find())

# Convierte los datos a un DataFrame de Pandas
df_from_mongo = pd.DataFrame(data_from_mongo)

# Representa los datos, por ejemplo, usando un gráfico de barras
df_from_mongo["prediction"].value_counts().plot(kind="bar")
plt.title("Predicciones")
plt.xlabel("Clase")
plt.ylabel("Frecuencia")
plt.show()

In [ ]:
----------------------------------------------------------------------------------------------------------------------------------------
# Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count

# Crear una sesión de Spark
spark = (
    SparkSession.builder.appName("MongoDBIntegration")
    .config("spark.mongodb.input.uri",
    "mongodb://mongo/grandes_volumenes.actividad3")
    .config("spark.mongodb.output.uri",
    "mongodb://mongo/grandes_volumenes.actividad3")
    .config(
    "spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:10.2.1")
    .getOrCreate()
    )

# Cargar datos desde MongoDB
# (asegúrate de configurar la conexión MongoDB correctamente)
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("database", "tu_base_de_datos").option("collection", "tu_coleccion").load()

# Agrupar datos por partido, equipo y acción, y contar
result_df = (
    df.groupBy("gameID", "team", "actions")
    .agg(count("*").alias("action_count"))
)

# Mostrar el resultado
result_df.show(truncate=False)

# Puedes guardar este DataFrame resultante en MongoDB si es necesario
# result_df.write.format("com.mongodb.spark.sql.DefaultSource").option("database", "tu_base_de_datos").option("collection", "resultados").save()

# Detener la sesión de Spark
spark.stop()

In [ ]:
----------------------------------------------------------------------------------------------------------------------------------------
#scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Supongamos que tienes un DataFrame 'result_df' con columnas como 'gameID', 'team', 'action_count', 'actions', y 'winner'.

# Convertir columnas categóricas a numéricas
le = LabelEncoder()
result_df['actions'] = le.fit_transform(result_df['actions'])
result_df['team'] = le.fit_transform(result_df['team'])
result_df['winner'] = le.fit_transform(result_df['winner'])

# Dividir los datos en conjunto de entrenamiento y prueba
X = result_df[['gameID', 'team', 'action_count']]
y = result_df['winner']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar un modelo
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predictions = model.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

In [ ]:

----------------------------------------------------------------------------------------------------------------------------------------
#Mongo
# Puedes guardar las predicciones en MongoDB utilizando PyMongo
from pymongo import MongoClient


client = MongoClient('mongodb://localhost:27019/')
db = client['tu_base_de_datos']
collection = db['predicciones']

# Supongamos que 'predictions' es una lista de predicciones que quieres guardar
pred_data = [{'gameID': game_id, 'prediction': pred} for game_id, pred in zip(X_test['gameID'], predictions)]
collection.insert_many(pred_data)

In [ ]:
----------------------------------------------------------------------------------------------------------------------------------------
#Jupyter
# Instala las bibliotecas necesarias en tu entorno de Jupyter
# pip install matplotlib pandas pymongo

import pandas as pd
import matplotlib.pyplot as plt

# Conecta con MongoDB y carga los datos
client = MongoClient('mongodb://localhost:27017/')
db = client['tu_base_de_datos']
collection = db['predicciones']
data_from_mongo = list(collection.find())

# Convierte los datos a un DataFrame de Pandas
df_from_mongo = pd.DataFrame(data_from_mongo)

# Representa los datos, por ejemplo, usando un gráfico de barras
df_from_mongo['prediction'].value_counts().plot(kind='bar')
plt.title('Predicciones')
plt.xlabel('Clase')
plt.ylabel('Frecuencia')
plt.show()